In [1]:
import jieba
import numpy as np
import pandas as pd

pd.read_csv("question_file.csv")

# 项目级别文件导入模版

In [2]:
import sys
from os.path import expanduser
from os import sep
project = "{}{}{}".format(expanduser("~"),sep,"linux_tools_for_chinese")
sys.path.append(project)
from file_tools import srcdir,algorithm_dir,search_model_dir,bash_functional_script_dir,python_functional_script_dir,question_data_dir
sys.path.append(algorithm_dir)
from algorithm.tfidf import IDF ,TF,TFIDF

In [3]:
srcdir,algorithm_dir,search_model_dir,bash_functional_script_dir,python_functional_script_dir,question_data_dir

('/home/manman/linux_tools_for_chinese/src/',
 '/home/manman/linux_tools_for_chinese/algorithm/',
 '/home/manman/linux_tools_for_chinese/search_model/',
 '/home/manman/linux_tools_for_chinese/bash_functional_script/',
 '/home/manman/linux_tools_for_chinese/python_functional_script/',
 '/home/manman/linux_tools_for_chinese/question_data/')

# 数据预处理模块

## 读取DF表格并选择dict结构

In [4]:
def change_list_from_DataFrame_func(table_path,column_names=[]):
    table = pd.read_csv(table_path)
    columns_list = [column for column in table.columns if 'Unnamed' in column]
    newtable = table.drop(columns_list,axis=1)
    newtable.to_csv(table_path)
    dicts = newtable.to_dict()
    return {column_name : [dicts[column_name][row] for row in dicts[column_name]] for column_name in column_names}
        
change_list_from_DataFrame_func("question_file.csv",["文字说明","命令"])

{'文字说明': ['扫描本机所在网段上有哪些主机是存活的',
  '端口扫描：输入目标主机ip，扫描某台主机开放了哪些端口',
  '隐藏扫描，输入目标主机ip，只在目标主机上留下很少的日志信息',
  'UDP端口扫描：输入目标主机ip，扫描目标主机开放了哪些UDP端口',
  '操作系统识别：输入目标主机ip，查询是哪个系统',
  '上传或者同步大型项目文件到服务器',
  '检查本机网段内ip',
  '查看本机网段内 激活/在线 的设备',
  '查询本地公网ip',
  '上传《网络工具》项目到GPU服务器',
  '上传《网络工具》项目到华为服务器',
  '查询系统运行时间',
  '查询系统开机时间',
  '查询系统历史启动时间',
  '存储盘的位置',
  '酒店开房数据集的位置'],
 '命令': ['sudo nmap -sP ',
  'sudo nmap -sT ',
  'sudo nmap -sS ',
  'sudo nmap -sU ',
  'sudo nmap -sS -O  ',
  'rsync -avP <文件位置> <服务器ip>:<服务器内待存位置>',
  'ifconfig | grep inet | grep netmask | grep b',
  'fping -g <内网ip>/24',
  'curl cip.cc',
  'rsync -avP /Users/zhangjing/网络工具/ GPUserver:/home/manman/网络工具/',
  'rsync -avP /Users/zhangjing/网络工具/ huawei:/root/网络工具/',
  'uptime -p / cat /proc/uptime| awk -F. \'{run_days=$1 / 86400;run_hour=($1 % 86400)/3600;run_minute=($1 % 3600)/60;run_second=$1 % 60;printf("系统已运行：%d天%d时%d分%d秒",run_days,run_hour,run_minute,run_second)}\'',
  'who -r / date -d "$(awk -F. \'{print $1}\' /proc/uptime) 

In [5]:
instructions_text = change_list_from_DataFrame_func("question_file.csv",['文字说明'])['文字说明']
command_encoder = change_list_from_DataFrame_func("question_file.csv",['命令'])['命令']
instructions_text,command_encoder

(['扫描本机所在网段上有哪些主机是存活的',
  '端口扫描：输入目标主机ip，扫描某台主机开放了哪些端口',
  '隐藏扫描，输入目标主机ip，只在目标主机上留下很少的日志信息',
  'UDP端口扫描：输入目标主机ip，扫描目标主机开放了哪些UDP端口',
  '操作系统识别：输入目标主机ip，查询是哪个系统',
  '上传或者同步大型项目文件到服务器',
  '检查本机网段内ip',
  '查看本机网段内 激活/在线 的设备',
  '查询本地公网ip',
  '上传《网络工具》项目到GPU服务器',
  '上传《网络工具》项目到华为服务器',
  '查询系统运行时间',
  '查询系统开机时间',
  '查询系统历史启动时间',
  '存储盘的位置',
  '酒店开房数据集的位置'],
 ['sudo nmap -sP ',
  'sudo nmap -sT ',
  'sudo nmap -sS ',
  'sudo nmap -sU ',
  'sudo nmap -sS -O  ',
  'rsync -avP <文件位置> <服务器ip>:<服务器内待存位置>',
  'ifconfig | grep inet | grep netmask | grep b',
  'fping -g <内网ip>/24',
  'curl cip.cc',
  'rsync -avP /Users/zhangjing/网络工具/ GPUserver:/home/manman/网络工具/',
  'rsync -avP /Users/zhangjing/网络工具/ huawei:/root/网络工具/',
  'uptime -p / cat /proc/uptime| awk -F. \'{run_days=$1 / 86400;run_hour=($1 % 86400)/3600;run_minute=($1 % 3600)/60;run_second=$1 % 60;printf("系统已运行：%d天%d时%d分%d秒",run_days,run_hour,run_minute,run_second)}\'',
  'who -r / date -d "$(awk -F. \'{print $1}\' /proc/uptime) second ago" +"

## 连接上下文

In [6]:
def context_func(paper_list):
    return "".join(paper_list)
context_func(instructions_text)

'扫描本机所在网段上有哪些主机是存活的端口扫描：输入目标主机ip，扫描某台主机开放了哪些端口隐藏扫描，输入目标主机ip，只在目标主机上留下很少的日志信息UDP端口扫描：输入目标主机ip，扫描目标主机开放了哪些UDP端口操作系统识别：输入目标主机ip，查询是哪个系统上传或者同步大型项目文件到服务器检查本机网段内ip查看本机网段内 激活/在线 的设备查询本地公网ip上传《网络工具》项目到GPU服务器上传《网络工具》项目到华为服务器查询系统运行时间查询系统开机时间查询系统历史启动时间存储盘的位置酒店开房数据集的位置'

## 加载标准停用词（标点符号）

In [7]:
import zhon
from zhon import hanzi
from zhon import pinyin
stopowrds = list(hanzi.stops)+list(hanzi.non_stops)+list(pinyin.stops)+list(pinyin.non_stops)

## 正则提取集合

In [8]:
import re

def extract_english(original):
    '''
    只要英文
    '''
    return re.findall(r'[a-zA-Z]+', original)

def extract_chinese(original):
    '''
    只要中文
    '''
    return re.findall(u'[\u4e00-\u9fa5]',original)

def extract_numbers(original):
    '''
    只要数字
    '''
    return re.findall(r'\d+',original)

def temp_extract(strings,symbles=stopowrds):
    '''
    临时删除
    '''
    srcrep = {i:'' for i in symbles }
    rep = dict((re.escape(k), v) for k, v in srcrep.items())
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], strings)

## 中英文字符判断

In [9]:
def is_chinese(uchar):
        """
        判断一个unicode是否是汉字
        """
        if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
                return True
        else:
                return False

def is_number(uchar):
        """
        判断一个unicode是否是数字
        """
        if uchar >= u'\u0030' and uchar<=u'\u0039':
                return True
        else:
                return False

def is_alphabet(uchar):
        """
        判断一个unicode是否是英文字母
        """
        if (uchar >= u'\u0041' and uchar<=u'\u005a') or (uchar >= u'\u0061' and uchar<=u'\u007a'):
                return True
        else:
                return False

def is_other(uchar):
        """
        判断是否非汉字，数字和英文字符
        """
        if not (is_chinese(uchar) or is_number(uchar) or is_alphabet(uchar)):
                return True
        else:
                return False
            
is_chinese("韩"),is_number("1"),is_alphabet("chinese"),is_other(",")

(True, True, True, True)

# 不同字符类型分割

In [10]:
def split_character_type(original):
    '''
    不同字符类型分割
    '''
    make = [0]
    diff = []
    n = 0
    temp = ""
    for char in original:
        if is_chinese(char):
            n = 0
        elif is_number(char):
            n = 1
        elif is_alphabet(char):
            n = 2
        elif is_other(char):
            n = 3
        else:
            n = 4
        make.append(n)
        if (make[-1]-make[-2]) == 0:
            diff.append(char)
        else:
            diff.append("|")
            diff.append(char)      
    return "".join(diff).split("|")
            
split_character_type(context_func(instructions_text))

['扫描本机所在网段上有哪些主机是存活的端口扫描',
 '：',
 '输入目标主机',
 'ip',
 '，',
 '扫描某台主机开放了哪些端口隐藏扫描',
 '，',
 '输入目标主机',
 'ip',
 '，',
 '只在目标主机上留下很少的日志信息',
 'UDP',
 '端口扫描',
 '：',
 '输入目标主机',
 'ip',
 '，',
 '扫描目标主机开放了哪些',
 'UDP',
 '端口操作系统识别',
 '：',
 '输入目标主机',
 'ip',
 '，',
 '查询是哪个系统上传或者同步大型项目文件到服务器检查本机网段内',
 'ip',
 '查看本机网段内',
 ' ',
 '激活',
 '/',
 '在线',
 ' ',
 '的设备查询本地公网',
 'ip',
 '上传',
 '《',
 '网络工具',
 '》',
 '项目到',
 'GPU',
 '服务器上传',
 '《',
 '网络工具',
 '》',
 '项目到华为服务器查询系统运行时间查询系统开机时间查询系统历史启动时间存储盘的位置酒店开房数据集的位置']

## 中文分词

In [11]:
def split_words_from_chinese_func(original):
    '''
    中文分词
    '''
    return list(jieba.cut(context_func(original)))
split_words = split_words_from_chinese_func(instructions_text[9])
split_words

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.480 seconds.
Prefix dict has been built successfully.


['上传', '《', '网络工具', '》', '项目', '到', 'GPU', '服务器']

## 英文分词

In [12]:
def split_words_from_english_func(original):
    '''
    英文空格分词
    '''
    return original.split()

## 过滤停用词

In [13]:
def filter_stop_words(words_list,stop_words_dict = stopowrds):
    '''
    过滤停用词
    '''
    return [word for word in words_list if word not in stop_words_dict ]
filter_stop_words(split_words)

['上传', '网络工具', '项目', '到', 'GPU', '服务器']

## 英文大小写转换

In [14]:
def case_conversion(msg,key=None):
    '''
    英文转换
    '''
    if key==None:
        return None
    return {"upper":msg.upper()  #upper()函数，将所有字母都转换成大写
     ,"lower":msg.lower()  #lower()函数，将所有字母都转换成小写
     ,"capitalize":msg.capitalize()  #capitalize()函数，将首字母都转换成大写，其余小写
     ,"title":msg.title()}[key] # 所有分隔英文单词首字母大写

case_conversion('www.什么BAIDU.com.123哈哈',"title")

'Www.什么Baidu.Com.123哈哈'

## ifidf 转句向量

In [15]:
sys.path.append(algorithm_dir)
from algorithm.tfidf import IDF,TF,TFIDF

## 保存句向量模型

## 生成倒排表

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.443 seconds.
Prefix dict has been built successfully.


['扫描',
 '本',
 '机',
 '所在',
 '网段',
 '上',
 '有',
 '哪些',
 '主机',
 '是',
 '存活',
 '的',
 '端口扫描',
 '：',
 '输入',
 '目标',
 '主机',
 'ip',
 '，',
 '扫描',
 '某台',
 '主机',
 '开放',
 '了',
 '哪些',
 '端口',
 '隐藏',
 '扫描',
 '，',
 '输入',
 '目标',
 '主机',
 'ip',
 '，',
 '只',
 '在',
 '目标',
 '主机',
 '上',
 '留下',
 '很少',
 '的',
 '日志',
 '信息',
 'UDP',
 '端口扫描',
 '：',
 '输入',
 '目标',
 '主机',
 'ip',
 '，',
 '扫描',
 '目标',
 '主机',
 '开放',
 '了',
 '哪些',
 'UDP',
 '端口',
 '操作系统',
 '识别',
 '：',
 '输入',
 '目标',
 '主机',
 'ip',
 '，',
 '查询',
 '是',
 '哪个',
 '系统',
 '上传',
 '或者',
 '同步',
 '大型项目',
 '文件',
 '到',
 '服务器',
 '检查',
 '本',
 '机',
 '网段',
 '内',
 'ip',
 '查看',
 '本',
 '机',
 '网段',
 '内',
 ' ',
 '激活',
 '/',
 '在线',
 ' ',
 '的',
 '设备',
 '查询',
 '本地',
 '公网',
 'ip',
 '上传',
 '《',
 '网络工具',
 '》',
 '项目',
 '到',
 'GPU',
 '服务器',
 '上传',
 '《',
 '网络工具',
 '》',
 '项目',
 '到',
 '华为',
 '服务器',
 '查询',
 '系统',
 '运行',
 '时间',
 '查询',
 '系统',
 '开机',
 '时间',
 '查询',
 '系统',
 '历史',
 '启动',
 '时间',
 '存储',
 '盘',
 '的',
 '位置',
 '酒店',
 '开房',
 '数据',
 '集',
 '的',
 '位置']

## 保存倒排表